<a href="https://colab.research.google.com/github/iammawaistariq/PAI-Lab/blob/main/Lab_6_EDA%2BSklearn_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Preparation

*   Feature Scaling
*   Feature Binning
*   Column Transformer
*   Function Transfoerm
*   Sklearn Pipeline

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.pipeline import Pipeline

In [4]:
titanic = pd.read_csv('titanic.csv')

**Feature Scaling**

In [ ]:
#Select features for scaling
features_to_scale = ['Age', 'Fare']

In [ ]:
# Handle missing values
# Fill missing values in 'Age' and 'Fare' with the median
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].median())
titanic['Fare'] = titanic['Fare'].fillna(titanic['Fare'].median())
#titanic['Age'].fillna(titanic['Age'].median(), inplace=True)
#titanic['Fare'].fillna(titanic['Fare'].median(), inplace=True)

In [ ]:
#Apply MinMaxScaler to the selected features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(titanic[features_to_scale])
#print(scaled_features)
print(type(scaled_features))

<class 'numpy.ndarray'>


In [ ]:

# Convert scaled features into a DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features_to_scale)
print(scaled_df)


          Age      Fare
0    0.271174  0.014151
1    0.472229  0.139136
2    0.321438  0.015469
3    0.434531  0.103644
4    0.434531  0.015713
..        ...       ...
886  0.334004  0.025374
887  0.233476  0.058556
888  0.346569  0.045771
889  0.321438  0.058556
890  0.396833  0.015127

[891 rows x 2 columns]


In [ ]:
#Concatenate the scaled features back into the original DataFrame
print(titanic[features_to_scale])
titanic[features_to_scale] = scaled_df
print(titanic[features_to_scale])
print(type(titanic[features_to_scale]))

      Age     Fare
0    22.0   7.2500
1    38.0  71.2833
2    26.0   7.9250
3    35.0  53.1000
4    35.0   8.0500
..    ...      ...
886  27.0  13.0000
887  19.0  30.0000
888  28.0  23.4500
889  26.0  30.0000
890  32.0   7.7500

[891 rows x 2 columns]
          Age      Fare
0    0.271174  0.014151
1    0.472229  0.139136
2    0.321438  0.015469
3    0.434531  0.103644
4    0.434531  0.015713
..        ...       ...
886  0.334004  0.025374
887  0.233476  0.058556
888  0.346569  0.045771
889  0.321438  0.058556
890  0.396833  0.015127

[891 rows x 2 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
# Step 6: Display the results
print("After MinMax Scaling:")
print(titanic.shape)
print(titanic.head())

After MinMax Scaling:
(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex       Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  0.271174      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  0.472229      1   
2                             Heikkinen, Miss. Laina  female  0.321438      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  0.434531      1   
4                           Allen, Mr. William Henry    male  0.434531      0   

   Parch            Ticket      Fare Cabin Embarked  
0      0         A/5 21171  0.014151   NaN        S  
1      0          PC 17599  0.139136   C85        C  
2      0  STON/O2. 3101282  0.015469   NaN        S  
3      0            113803  0.103644  C123      

# **Feature Binning**

In [ ]:
# Extract Title from Name
titanic['Title'] = titanic['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
print(titanic['Title'])

0        Mr
1       Mrs
2      Miss
3       Mrs
4        Mr
       ... 
886     Rev
887    Miss
888    Miss
889      Mr
890      Mr
Name: Title, Length: 891, dtype: object


In [ ]:
# Define title mapping dictionary
title_mapping = {
    'Mr': 'Mr', 'Mrs': 'Mrs', 'Miss': 'Miss', 'Master': 'Master',
    'Capt': 'Officer', 'Col': 'Officer', 'Major': 'Officer', 'Dr': 'Officer', 'Rev': 'Officer',
    'Countess': 'Royalty', 'Sir': 'Royalty', 'Lady': 'Royalty', 'Jonkheer': 'Royalty', 'Don': 'Royalty',
    'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'
}
print(type(titanic['Title']))
# Map extracted titles to broader categories & handle unknown titles
titanic['Title_Group'] = titanic['Title'].map(title_mapping)
print(titanic)
#filtered_titanic = titanic[titanic['Title_Group'] == 'Other'].tail(100)
#print(filtered_titanic)

<class 'pandas.core.series.Series'>
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex       Age  \
0                              Braund, Mr. Owen Harris    male  0.271174   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  0.472229   
2                               Heikkinen, Miss. Laina  female  0.321438   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  0.434531   
4                             Allen, Mr. William Henry    male  0.434531   
..                                                 

In [ ]:
# Drop the original 'Title' column (optional)
titanic.drop(columns=['Title'], inplace=True)
print(titanic.head())

# Encode the categorical feature using One-Hot Encoding (optional for ML models)
titanic = pd.get_dummies(titanic, columns=['Title_Group'],  dtype=int)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex       Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  0.271174      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  0.472229      1   
2                             Heikkinen, Miss. Laina  female  0.321438      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  0.434531      1   
4                           Allen, Mr. William Henry    male  0.434531      0   

   Parch            Ticket      Fare Cabin Embarked Title_Group  
0      0         A/5 21171  0.014151   NaN        S          Mr  
1      0          PC 17599  0.139136   C85        C         Mrs  
2      0  STON/O2. 3101282  0.015469   NaN        S        Miss  
3      0            113803  0.10

In [ ]:
# Display the updated dataset with new feature
print(titanic.head())
print(titanic.shape)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex       Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  0.271174      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  0.472229      1   
2                             Heikkinen, Miss. Laina  female  0.321438      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  0.434531      1   
4                           Allen, Mr. William Henry    male  0.434531      0   

   Parch            Ticket      Fare Cabin Embarked  Title_Group_Master  \
0      0         A/5 21171  0.014151   NaN        S                   0   
1      0          PC 17599  0.139136   C85        C                   0   
2      0  STON/O2. 3101282  0.015469   NaN        S                   0

**Column Transformer**

In [ ]:
# Sample DataFrame
data = pd.DataFrame({
    'Age': [22, None, 24, 22, None, 24],
    'Sex': ['male', 'female', 'female','male', 'female', 'female'],
    'Embarked': ['B', 'B', 'C', 'C', 'C','S'],
    'Fare': [7.25, 71.83, 8.05,7.25, 71.83, 8.05]
})

print("Original DataFrame:")
print(data)

# Define ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('age_imputer', SimpleImputer(strategy='mean'), ['Age']),
    ('ohe', OneHotEncoder(), ['Sex','Embarked'])
], remainder='passthrough')  # Pass through other columns like 'Fare'

# Transform the data
transformed_data = preprocessor.fit_transform(data)

# Output shape and transformed data
print("\nTransformed Data Shape:")
print(transformed_data.shape)
#print(type(transformed_data))


transformed_df = pd.DataFrame(transformed_data)
print("\nTransformed DataFrame:")
print(transformed_df)

Original DataFrame:
    Age     Sex Embarked   Fare
0  22.0    male        B   7.25
1   NaN  female        B  71.83
2  24.0  female        C   8.05
3  22.0    male        C   7.25
4   NaN  female        C  71.83
5  24.0  female        S   8.05

Transformed Data Shape:
(6, 7)

Transformed DataFrame:
      0    1    2    3    4    5      6
0  22.0  0.0  1.0  1.0  0.0  0.0   7.25
1  23.0  1.0  0.0  1.0  0.0  0.0  71.83
2  24.0  1.0  0.0  0.0  1.0  0.0   8.05
3  22.0  0.0  1.0  0.0  1.0  0.0   7.25
4  23.0  1.0  0.0  0.0  1.0  0.0  71.83
5  24.0  1.0  0.0  0.0  0.0  1.0   8.05


**Function Transfomer**

In [ ]:
# Sample DataFrame
data = pd.DataFrame({
    'Age': [22, None, 24,22, None, 24],
    'Sex': ['male', 'female', 'female','male', 'female', 'female'],
    'Embarked': ['B', 'B', 'C', None, 'C','S'],
    'Fare': [7.25, 71.83, 8.05,7.25, 71.83, 8.05]
})

print("Original DataFrame:")
print(data)

# Custom function to impute 'Embarked'
def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    print(X['Embarked'])
    return X

# Define ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('age_imputer', SimpleImputer(strategy='mean'), ['Age']),
    ('embarked_imputer', FunctionTransformer(impute_embarked), ['Embarked']),
    ('ohe', OneHotEncoder(), ['Sex','Embarked'])
], remainder='passthrough')  # Pass through other columns like 'Fare'

# Transform the data
transformed_data = preprocessor.fit_transform(data)

# Output shape and transformed data
print("\nTransformed Data Shape:")
print(transformed_data.shape)
print(transformed_data)


transformed_df = pd.DataFrame(transformed_data)
print("\nTransformed DataFrame:")
print(transformed_df)


Original DataFrame:
    Age     Sex Embarked   Fare
0  22.0    male        B   7.25
1   NaN  female        B  71.83
2  24.0  female        C   8.05
3  22.0    male     None   7.25
4   NaN  female        C  71.83
5  24.0  female        S   8.05
0    B
1    B
2    C
3    B
4    C
5    S
Name: Embarked, dtype: object

Transformed Data Shape:
(6, 9)
[[22.0 'B' 0.0 1.0 1.0 0.0 0.0 0.0 7.25]
 [23.0 'B' 1.0 0.0 1.0 0.0 0.0 0.0 71.83]
 [24.0 'C' 1.0 0.0 0.0 1.0 0.0 0.0 8.05]
 [22.0 'B' 0.0 1.0 0.0 0.0 0.0 1.0 7.25]
 [23.0 'C' 1.0 0.0 0.0 1.0 0.0 0.0 71.83]
 [24.0 'S' 1.0 0.0 0.0 0.0 1.0 0.0 8.05]]

Transformed DataFrame:
      0  1    2    3    4    5    6    7      8
0  22.0  B  0.0  1.0  1.0  0.0  0.0  0.0   7.25
1  23.0  B  1.0  0.0  1.0  0.0  0.0  0.0  71.83
2  24.0  C  1.0  0.0  0.0  1.0  0.0  0.0   8.05
3  22.0  B  0.0  1.0  0.0  0.0  0.0  1.0   7.25
4  23.0  C  1.0  0.0  0.0  1.0  0.0  0.0  71.83
5  24.0  S  1.0  0.0  0.0  0.0  1.0  0.0   8.05


**Sklean Pipeline**

In [ ]:
# Sample DataFrame
data = pd.DataFrame({
    'Age': [22, None, 24,22, None, 24],
    'Sex': ['male', 'female', 'female','male', 'female', 'female'],
    'Embarked': ['B', 'B', 'C', None, 'C','S'],
    'Fare': [7.25, 71.83, 8.05,7.25, 71.83, 8.05]
})

print("Original DataFrame:")
print(data)

# Custom function to impute 'Embarked'
def impute_embarked(X):
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])  # Fill missing values
    print (X['Embarked'])
    return X

preprocessor = ColumnTransformer(transformers=[
    ('age_imputer', SimpleImputer(strategy='mean'), ['Age']),
    ('embarked_encoder', Pipeline(steps=[
        ('imputer', FunctionTransformer(impute_embarked)),  # Impute Embarked first
        ('onehot', OneHotEncoder())  # Then apply OneHotEncoder
    ]), ['Embarked']),
    ('ohe', OneHotEncoder(), ['Sex'])
], remainder='passthrough')  # Pass through other columns like 'Fare'


# Transform the data
transformed_data = preprocessor.fit_transform(data)

# Output shape and transformed data
print("\nTransformed Data Shape:")
print(transformed_data.shape)
print(transformed_data)

transformed_df = pd.DataFrame(transformed_data)
print("\nTransformed DataFrame:")
print(transformed_df)


Original DataFrame:
    Age     Sex Embarked   Fare
0  22.0    male        B   7.25
1   NaN  female        B  71.83
2  24.0  female        C   8.05
3  22.0    male     None   7.25
4   NaN  female        C  71.83
5  24.0  female        S   8.05
0    B
1    B
2    C
3    B
4    C
5    S
Name: Embarked, dtype: object

Transformed Data Shape:
(6, 7)
[[22.    1.    0.    0.    0.    1.    7.25]
 [23.    1.    0.    0.    1.    0.   71.83]
 [24.    0.    1.    0.    1.    0.    8.05]
 [22.    1.    0.    0.    0.    1.    7.25]
 [23.    0.    1.    0.    1.    0.   71.83]
 [24.    0.    0.    1.    1.    0.    8.05]]

Transformed DataFrame:
      0    1    2    3    4    5      6
0  22.0  1.0  0.0  0.0  0.0  1.0   7.25
1  23.0  1.0  0.0  0.0  1.0  0.0  71.83
2  24.0  0.0  1.0  0.0  1.0  0.0   8.05
3  22.0  1.0  0.0  0.0  0.0  1.0   7.25
4  23.0  0.0  1.0  0.0  1.0  0.0  71.83
5  24.0  0.0  0.0  1.0  1.0  0.0   8.05


In [5]:
import pandas as pd

# Create the FamilySize feature
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch'] + 1

# Display the new feature with related columns
print(titanic[['SibSp', 'Parch', 'FamilySize']].head())


   SibSp  Parch  FamilySize
0      1      0           2
1      1      0           2
2      0      0           1
3      1      0           2
4      0      0           1


In [6]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer
from sklearn.impute import SimpleImputer

# Step 1: Custom FunctionTransformer to create FamilySize
def add_family_size(X):
    X = X.copy()
    X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
    return X

# Step 2: Function to impute Embarked (for use in pipeline)
def impute_embarked(X):
    X = X.copy()
    X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])
    return X

# Step 3: Define preprocessing pipeline using ColumnTransformer
# Features involved
numeric_features = ['Age', 'Fare', 'FamilySize']
categorical_features = ['Sex', 'Embarked']

# Pipeline for numeric columns
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())
])

# Pipeline for categorical columns
categorical_pipeline = Pipeline(steps=[
    ('embarked_imputer', FunctionTransformer(impute_embarked, validate=False)),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Step 4: Overall ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_pipeline, numeric_features),
    ('cat', categorical_pipeline, categorical_features)
])

# Step 5: Full pipeline including custom feature creation
full_pipeline = Pipeline(steps=[
    ('family_size_creator', FunctionTransformer(add_family_size, validate=False)),
    ('preprocessing', preprocessor)
])

# Step 6: Select columns needed for the pipeline
selected_columns = ['Age', 'Fare', 'Sex', 'Embarked', 'SibSp', 'Parch']

# Step 7: Apply the pipeline
transformed_data = full_pipeline.fit_transform(titanic[selected_columns])

# Step 8: Convert transformed data into a DataFrame
# Get column names from encoders
ohe_sex = full_pipeline.named_steps['preprocessing'].named_transformers_['cat'].named_steps['encoder']
encoded_columns = ohe_sex.get_feature_names_out(categorical_features)

# Combined column names
final_columns = ['Age', 'Fare', 'FamilySize'] + list(encoded_columns)

# Create DataFrame
transformed_df = pd.DataFrame(transformed_data, columns=final_columns)

# Display result
print("\nTransformed Titanic Dataset:")
print(transformed_df.head())
print("Shape:", transformed_df.shape)



Transformed Titanic Dataset:
        Age      Fare  FamilySize  Sex_female  Sex_male  Embarked_C  \
0  0.271174  0.014151         0.1         0.0       1.0         0.0   
1  0.472229  0.139136         0.1         1.0       0.0         1.0   
2  0.321438  0.015469         0.0         1.0       0.0         0.0   
3  0.434531  0.103644         0.1         1.0       0.0         0.0   
4  0.434531  0.015713         0.0         0.0       1.0         0.0   

   Embarked_Q  Embarked_S  
0         0.0         1.0  
1         0.0         0.0  
2         0.0         1.0  
3         0.0         1.0  
4         0.0         1.0  
Shape: (891, 8)


# Lab Task

**Apply Feature Creation preprocessing step on the Titanic dataset to create a Family Size feature which calculates Family Size for each Passenger using following Equation. FamilySize = SibSp + Parch + 1**

**Apply ColumnTransformer, FunctionTransformer and Sklearn Pipeline on the Titanic dataset.**

**Use ColumnTransformer, FunctionTransformer and Sklearn Pipeline features to preprocess the following dataset**

https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease

# Heart Disease Dataset

In [16]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


In [7]:
dataset = pd.read_csv('/content/heart_2020_cleaned.csv')

In [8]:
dataset.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [9]:
dataset.tail()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No
319794,No,46.56,No,No,No,0.0,0.0,No,Female,80 or older,Hispanic,No,Yes,Good,8.0,No,No,No


In [10]:
dataset.shape

(319795, 18)

In [13]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [15]:
dataset.isnull().sum()

,0
HeartDisease,0
BMI,0
Smoking,0
AlcoholDrinking,0
Stroke,0
PhysicalHealth,0
MentalHealth,0
DiffWalking,0
Sex,0
AgeCategory,0


In [21]:
# Separate features and target
X = dataset.drop('HeartDisease', axis=1)
y = dataset['HeartDisease'].apply(lambda x: 1 if x == 'Yes' else 0)


In [23]:
# Identify column types
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=np.number).columns.tolist()

# Define a FunctionTransformer to perform any custom transformations (e.g., BMI binning as an example)
def custom_bmi_transform(X_df):
    X_df = X_df.copy()
    if 'BMI' in X_df.columns:
        X_df['BMI_category'] = pd.cut(X_df['BMI'], bins=[0, 18.5, 24.9, 29.9, 100], labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    return X_df

bmi_transformer = FunctionTransformer(custom_bmi_transform, validate=False)

# Update feature list after custom transformation
# This will be done inside a pipeline step, so ColumnTransformer should reflect final step columns

# Preprocessing for numerical features
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine all transformations
preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

# Full pipeline including custom function and final preprocessing
full_pipeline = Pipeline([
    ('custom_bmi', bmi_transformer),
    ('preprocessor', preprocessor)
])

# Fit and transform the data
X_preprocessed = full_pipeline.fit_transform(X)

# Train/test split for modeling
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Final check
print("Preprocessing complete. Transformed feature shape:", X_train.shape)

Preprocessing complete. Transformed feature shape: (255836, 50)
